Python Packages

- pip install customtkinter

In [1]:
pip install customtkinter

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tkinter
import customtkinter
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import time
from queue import Queue
import threading

In [3]:
# Main Window
#===================================================================

customtkinter.set_appearance_mode("light")

root_ctk = customtkinter.CTk()
root_ctk.title('Path-Search Algorithm')

# Set the width and heigh of the window
root_ctk.geometry(f"{900}x{720}")

root_ctk.grid_columnconfigure(0, weight=0)
root_ctk.grid_columnconfigure((1,2), weight=1)

root_ctk.grid_rowconfigure((0,1,2,3,4,5,6,7), weight=1)
root_ctk.grid_rowconfigure((8,9,10), weight=0)

=====

In [4]:
# Speed of the simulation (in seconds)
speed = 2.0

# Simulation state True for play, False for pause
paused = False

# Event to control pause and play
flag = threading.Event()  

play_thread = None

indx = 0

# Fixed Graph
fixed_graph_state = True

# Selected algorithm 1-DFS 2-BFS 3-Beam etc...
algo_num = 1

# Graph
G = nx.DiGraph()

# Events / Functions

In [5]:
def bfs_path_search(graph, start, end):
    # Initialize a queue for BFS
    queue = Queue()
    
    # Enqueue the starting node along with an empty path and cost
    queue.put((start, [], 0))
    
    # Mark the starting node as visited
    visited = set([start])
    search_path = []
    
    while not queue.empty():
        # Dequeue a node and its path and cost
        current, path, cost = queue.get()
        
        # Explore neighbors of the current node
        for neighbor, weight in graph[current].items():
            
            if neighbor not in visited:
                # Enqueue the neighbor along with the updated path and cost
                queue.put((neighbor, path + [(current, neighbor)], cost + weight['weight']))
                
                # Mark the neighbor as visited
                visited.add(neighbor)
                
                # Add to search path (for simulation purposes)
                search_path.append((current, neighbor)) 
                
                if neighbor == end:
                    queue.put((neighbor, path + [(current, neighbor)], cost + weight['weight']))
                    return path + [(current,neighbor)], search_path
                
    # If no path is found
    return [], search_path

def dfs_path_search(graph, start_node, search_node):
    visited = set()
    stack = [start_node]
    node_track = [start_node]
    order = []
    path = []
    sim_path = []
    cur_node = start_node
    
    while stack:
        current_node = stack.pop()
        cur_node = node_track[-1]
        node_track.append(current_node)
            
        if current_node not in visited:
            order.append(current_node)
            visited.add(current_node)
            
            # Track the path for simulation purposes
            path.append((cur_node,current_node))
            sim_path.append((cur_node,current_node)) # For simulation
            
            # If search node was found
            if current_node == search_node:
                break
            
            # If node has no neighbor
            if not any(True for _ in graph.neighbors(current_node)):
                x = node_track.pop()
                stack.append(x)
                cur_node = node_track[-1]
                
            for neighbor in graph.neighbors(current_node):
                if neighbor not in visited:
                    stack.append(neighbor)
        else:
            has_all_visited_neighbor = True
            
            for neighbor in graph.neighbors(current_node):
                if neighbor not in visited:
                    has_all_visited_neighbor = False
            
            if has_all_visited_neighbor:       
                sim_path.append((cur_node, cur_node)) # For simulation  
                node_track.pop()
                x = node_track.pop()
                stack.append(x)
                cur_node = node_track[-1]
                
    return path, sim_path


# ============================================================================================================
# ============================================================================================================
# ============================================================================================================
# ============================================================================================================
# ============================================================================================================

# Set fixed graph
def set_fixed_graph(G):
    G.add_edge('A', 'B', weight=1)
    G.add_edge('A', 'C', weight=3)
    G.add_edge('B', 'F', weight=2)
    G.add_edge('B', 'H', weight=5)
    G.add_edge('C', 'H', weight=4)
    G.add_edge('C', 'D', weight=4)
    G.add_edge('D', 'I', weight=5)
    G.add_edge('D', 'E', weight=1)
    G.add_edge('H', 'J', weight=3)
    G.add_edge('F', 'K', weight=1)
    G.add_edge('K', 'G', weight=4)
    G.add_edge('K', 'O', weight=2)
    G.add_edge('I', 'M', weight=3)
    G.add_edge('I', 'O', weight=5)
    G.add_edge('E', 'L', weight=2)
    G.add_edge('L', 'M', weight=1)
    G.add_edge('L', 'P', weight=4)
    G.add_edge('J', 'K', weight=2)
    G.add_edge('M', 'N', weight=4)

    # Specify custom positions for nodes (X,Y)
    custom_positions = {
        'A': (-6, 6), 
        'B': (-6, 2), 
        'C': (-2, 6), 
        'D': (2, 6), 
        'E': (6, 6), 
        'F': (-6, -2), 
        'G': (-6, -6), 
        'H': (-4, 4), 
        'I': (0, 2), 
        'J': (-2, 0), 
        'K': (-4, -4), 
        'L': (6, 2), 
        'M': (3, 0), 
        'N': (3, -4), 
        'O': (0, -4), 
        'P': (6, -6)}
    
    return (G, custom_positions)

# Initialized default fixed graph
def display_init_graph(graph, positions):
    global fig, ax
    labels = nx.get_edge_attributes(graph, 'weight')
    plt.close()
    nx.draw(graph, pos=positions, with_labels=True, node_size=1000, node_color='#98ff98', font_size=14, font_weight="bold", ax=ax)
    nx.draw_networkx_edge_labels(graph, pos=positions, edge_labels=labels, ax=ax)
    root_ctk.label_nodes_count.configure(text = "No. of Nodes:   16" )
    root_ctk.label_path_count.configure(text = "No. of Paths:   19" )
    

# Visualise simulation by updating the color of the current node
def visualize_graph(graph, positions, current_node):
    global fig, ax
    labels = nx.get_edge_attributes(graph, 'weight')
    plt.close()
    nx.draw(graph, pos=positions, with_labels=True, node_size=1000, node_color=['#ea077c' if n == current_node else '#98ff98' for n in graph.nodes], font_size=14, font_weight="bold", ax=ax)
    nx.draw_networkx_edge_labels(graph, pos=positions, edge_labels=labels, ax=ax)
    

# Use to display the graph into the GUI
def embed_matplotlib_figure_to_tkinter():
    global canvas
    canvas.draw()
    root_ctk.update()
    plt.close()
    
#     return canvas

# Update the node color
def update_graph(graph, positions, current_node):
    visualize_graph(G, custom_positions, current_node)
    embed_matplotlib_figure_to_tkinter()

# Visualise simulation by updating the color of the current node
def visualize_graph_bfs(graph, positions, n1, n2):
    global fig, ax
    labels = nx.get_edge_attributes(graph, 'weight')
    plt.close()
    nx.draw(graph, pos=positions, with_labels=True, node_size=1000, 
            node_color=['#ea077c' if n == n2 else '#299617' 
                        if n == n1 else '#98ff98' 
                        for n in graph.nodes], 
            font_size=14, font_weight="bold", ax=ax)
    nx.draw_networkx_edge_labels(graph, pos=positions, edge_labels=labels, ax=ax)
    return fig    

def update_graph_bfs(graph, positions, n1, n2):
    visualize_graph_bfs(G, custom_positions, n1, n2)
    embed_matplotlib_figure_to_tkinter()

=====

In [6]:
# Start Simulation
# ============================================================================================================    
# Triggers when <Start> button is click
def start():
    global algo_num, G, flag, root_ctk, indx
    
    start = root_ctk.entry_start_node.get()
    end = root_ctk.entry_search_node.get()
    
    
    if start == "" or end == "":
        root_ctk.label_remarks.configure(text ="Please fill the fields <Start Node> and <Search Node>.")
        root_ctk.btn_start.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"), state="normal")
        root_ctk.btn_pause.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"), state="disabled")
        
    elif start == end:
        root_ctk.label_remarks.configure(text ="Start node and Search node should not be equal!")
        root_ctk.btn_start.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"), state="normal")
        root_ctk.btn_pause.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"), state="disabled")
        
    else:
        root_ctk.entry_search_node.configure(state="disabled")
        root_ctk.entry_start_node.configure(state="disabled")
        root_ctk.label_remarks.configure(text ="")
        root_ctk.btn_start.configure(fg_color = ("green","white"), hover_color = ("green","white"), state="disabled")
        root_ctk.btn_pause.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"), state="normal")
        
        
        # ================================================================================================
        # If the chosen algo is DFS
        
        if algo_num == 1:
            
#           # Disable algo button to avoid switch while simulating
            root_ctk.dfs_btn.configure(fg_color = ("green","white"), hover_color = ("green","white"), state="disabled")
            root_ctk.bfs_btn.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"), state="disabled")
            
            path, path_search = dfs_path_search(G, start, end)
            search_path = path
            search_path.remove((start,start))
            n1 = search_path[0][0]
            n2 = search_path[0][1]
            
            print(search_path)
            # Display count of enqueues and cost
            total_enqueues = len(search_path)
            total_cost = sum(G[edge[0]][edge[1]]['weight'] for edge in search_path)
            
            root_ctk.label_enqueues.configure(text = "Enqueues Count:  " + str(total_enqueues))
            root_ctk.label_path_cost.configure(text = "Path Cost:  " + str(total_cost))
            
            if indx >= len(path_search):
                indx = 0
            
            while not flag.is_set():
                # Check if index reach the end of the path list
                if indx == len(path_search):
                    if(n2 == end):
                        root_ctk.label_remarks.configure(text = "Node  [ " + str(end) + " ]  found!", text_color = "green")
                    else:
                        root_ctk.label_remarks.configure(text = "Node  [ " + str(end) + " ]  was not found!", text_color = "green")
                        
                    root_ctk.btn_start.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"), state="normal")
                    root_ctk.btn_pause.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"), state="disabled")
                    break
                    
                else:
                    n1 = path_search[indx][0]
                    n2 = path_search[indx][1]
                    
                    if n1 == n2:
                        root_ctk.label_remarks.configure(text = "", text_color = "green")
                    else:
                        root_ctk.label_remarks.configure(text = str(n1) + "  ----->  " + str(n2), text_color = "green")
                    
#                   # Update the graph
                    graph, positions = set_fixed_graph(G)
                    update_graph_bfs(graph, positions, start, n2)

                    time.sleep(speed)
                    indx += 1
            
            
        # ==================================================================================================
        # If the chosen algo is BFS
        elif algo_num == 2:
            
#           # Disable algo button to avoid switch while simulating
            root_ctk.bfs_btn.configure(fg_color = ("green","white"), hover_color = ("green","white"), state="disabled")
            root_ctk.dfs_btn.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"), state="disabled")
            
            shortest_path, search_path = bfs_path_search(G, start, end)
            
            n1 = search_path[0][0]
            n2 = search_path[0][1]
            
            total_enqueues = len(search_path)
            total_cost = sum(G[edge[0]][edge[1]]['weight'] for edge in search_path)
            
            root_ctk.label_enqueues.configure(text = "Enqueues Count:  " + str(total_enqueues))
            root_ctk.label_path_cost.configure(text = "Path Cost:  " + str(total_cost))
            
            if indx >= len(search_path):
                indx = 0
            
            while not flag.is_set():
                if indx == len(search_path):
                    if(n2 == end):
                        root_ctk.label_remarks.configure(text = "Node  [ " + str(end) + " ]  found!", text_color = "green")
                    else:
                        root_ctk.label_remarks.configure(text = "Node  [ " + str(end) + " ]  was not found!", text_color = "green")
                        
                    root_ctk.btn_start.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"), state="normal")
                    root_ctk.btn_pause.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"), state="disabled")
                    break
                    
                else:
                    n1 = search_path[indx][0]
                    n2 = search_path[indx][1]
                    
                    root_ctk.label_remarks.configure(text = str(n1) + "  ----->  " + str(n2), text_color = "green")
                    graph, positions = set_fixed_graph(G)
                    update_graph_bfs(graph, positions, n1, n2)

                    time.sleep(speed)
                    indx += 1
        
        root_ctk.btn_start.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"), state="normal")
        root_ctk.btn_pause.configure(fg_color = ("green", "white"), hover_color = ("green","white"), state="disabled")
        
        
        # Disable start and search node input when pause
        # Normalize if the simulation ends
        if flag.is_set():
            root_ctk.entry_search_node.configure(state="disabled")
            root_ctk.entry_start_node.configure(state="disabled")
#           # Disable algo button to avoid switch while simulating
            root_ctk.dfs_btn.configure(fg_color = ("green","white"), hover_color = ("green","white"), state="disabled")
            root_ctk.bfs_btn.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"), state="disabled")
        else:
            root_ctk.entry_search_node.configure(state="normal")
            root_ctk.entry_start_node.configure(state="normal")
#           # Disable algo button to avoid switch while simulating
            root_ctk.dfs_btn.configure(fg_color = ("green","white"), hover_color = ("green","white"), state="normal")
            root_ctk.bfs_btn.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"), state="normal")
    

# Pause Simulation
# ============================================================================================================    
# Triggers when <Pause> button is click
def pause():
    global paused, play_thread
    flag.set()
    paused = True
    
def resume():
    global paused, play_thread
    flag.clear()
    paused = False
    play_thread = threading.Thread(target=start)
    play_thread.start()

def play_clicked():
    global play_thread, paused
    
    if play_thread is None or not play_thread.is_alive() and not paused:
        play_thread = threading.Thread(target=start)
        play_thread.start()
        paused = False
    else:
        resume()
        
def pause_clicked():
    pause()

# Button DFS event click
# ============================================================================================================

def button_dfs():
    global algo_num, indx
    # Change the button color of the selected button
    root_ctk.dfs_btn.configure(fg_color = ("green","white"), hover_color = ("green","white"))
    root_ctk.bfs_btn.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"))
#     root_ctk.beam_btn.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"))
#     root_ctk.hill_btn.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"))
#     root_ctk.a_btn.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"))
    root_ctk.btn_start.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"))
    root_ctk.btn_pause.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"))
    algo_num = 1
    indx = 0
    

# Button BFS event click
# ============================================================================================================

def button_bfs():
    global algo_num, indx
    # Change the button color of the selected button
    root_ctk.dfs_btn.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"))
    root_ctk.bfs_btn.configure(fg_color = ("green","white"), hover_color = ("green","white"))
#     root_ctk.beam_btn.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"))
#     root_ctk.hill_btn.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"))
#     root_ctk.a_btn.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"))
    root_ctk.btn_start.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"))
    root_ctk.btn_pause.configure(fg_color = ("#ea077c", "white"), hover_color = ("green","white"))
    algo_num = 2
    indx = 0

    
# Set speed label according to slider value
# ============================================================================================================   
def set_speed_lbl(value):
    global speed
    speed = value
    root_ctk.label_sldr.configure(text = str(speed) + "sec")

=====

# Graphical User Interface Layout

In [7]:
# Frames


# Left Sidebar Frame
# ============================
root_ctk.left_sidebar_frame = customtkinter.CTkFrame(root_ctk, corner_radius=0)
root_ctk.left_sidebar_frame.grid(row=0, column=0, rowspan=10, sticky="nsew", padx =(5,5), pady=(5,5))
root_ctk.left_sidebar_frame.grid_rowconfigure(10, weight=1)
root_ctk.left_sidebar_frame.grid_columnconfigure(1, weight=1)

# Graph frame
# ============================

root_ctk.graph_frame = customtkinter.CTkFrame(root_ctk, corner_radius=0)
root_ctk.graph_frame.grid(row=0, column=1, columnspan=2, rowspan=7, sticky="nsew", padx = (0,5), pady=(5,0))
root_ctk.graph_frame.grid_columnconfigure(1, weight=1)
root_ctk.graph_frame.grid_rowconfigure(1, weight=1)


# Remarks frame
# ============================

root_ctk.remarks_frame = customtkinter.CTkFrame(root_ctk, corner_radius=0)
root_ctk.remarks_frame.grid(row=7, column=1, columnspan = 2, sticky="nsew", padx = (0,5), pady=(5,0))
root_ctk.remarks_frame.grid_columnconfigure((0), weight=1)
root_ctk.remarks_frame.grid_rowconfigure(0, weight=0)

# Output frame
# ============================

root_ctk.output_frame = customtkinter.CTkFrame(root_ctk, corner_radius=0)
root_ctk.output_frame.grid(row=8, column=1, sticky="nsew", padx = (0,0), pady=(5,0))
root_ctk.output_frame.grid_columnconfigure(0, weight=0)
root_ctk.output_frame.grid_rowconfigure((0,1), weight=0)

# Count frame
# ============================

root_ctk.count_frame = customtkinter.CTkFrame(root_ctk, corner_radius=0)
root_ctk.count_frame.grid(row=8, column=2, sticky="nsew", padx = (5,5), pady=(5,0))
root_ctk.count_frame.grid_columnconfigure((0,1), weight=1)
root_ctk.count_frame.grid_rowconfigure((0,1), weight=0)

# Bottom Frame
# ============================

root_ctk.bottom_frame = customtkinter.CTkFrame(root_ctk, corner_radius=0)
root_ctk.bottom_frame.grid(row=9, column=1, columnspan=2, sticky="nsew", padx = (0,5), pady=(5,5))
root_ctk.bottom_frame.grid_columnconfigure((0,2), weight=0)
root_ctk.bottom_frame.grid_columnconfigure((1,3,4), weight=1)
root_ctk.bottom_frame.grid_rowconfigure(0, weight=0)

=====

In [8]:
# Left Sidebar Elements
#===================================================================

# Label
root_ctk.logo_label = customtkinter.CTkLabel(root_ctk.left_sidebar_frame, text="Path-Search\nAlgorithm", font=customtkinter.CTkFont(size=20, weight="bold"))
root_ctk.logo_label.grid(row=0, column=0, padx=10, pady=(10, 10), sticky="nsew")


# Set start_node
root_ctk.label_input_val = customtkinter.CTkLabel(root_ctk.left_sidebar_frame, text = "Start Node: ", font=customtkinter.CTkFont(size=13, weight="bold"))
root_ctk.label_input_val.grid(row=1 , column=0, padx=10, pady=(20,0), sticky="w")

root_ctk.entry_start_node = customtkinter.CTkEntry(root_ctk.left_sidebar_frame, placeholder_text="Start Node")
root_ctk.entry_start_node.grid(row=2, column=0, padx=10, pady=(0,5), sticky="nsew")

# Set search_node
root_ctk.label_input_val1 = customtkinter.CTkLabel(root_ctk.left_sidebar_frame, text = "Search Node: ", font=customtkinter.CTkFont(size=13, weight="bold"))
root_ctk.label_input_val1.grid(row=3 , column=0, padx=10, pady=0, sticky="w")

root_ctk.entry_search_node = customtkinter.CTkEntry(root_ctk.left_sidebar_frame, placeholder_text="Search Node")
root_ctk.entry_search_node.grid(row=4, column=0, padx=10, pady=(0,20), sticky="nsew")

# DFS button
root_ctk.dfs_btn = customtkinter.CTkButton(root_ctk.left_sidebar_frame, text="Depth-First", font=customtkinter.CTkFont(weight="bold"), command=button_dfs)
root_ctk.dfs_btn.grid(row=5, column=0, padx=10, pady=5, sticky="nsew")
# root_ctk.dfs_btn.configure(fg_color=("#ea077c", "white") ,hover_color = ("green","white"))
root_ctk.dfs_btn.configure(fg_color = ("green","white"), hover_color = ("green","white"))

# BFS button
root_ctk.bfs_btn = customtkinter.CTkButton(root_ctk.left_sidebar_frame, text="Breadth-First", font=customtkinter.CTkFont(weight="bold"),command=button_bfs)
root_ctk.bfs_btn.grid(row=6, column=0, padx=10, pady=5, sticky="nsew")
root_ctk.bfs_btn.configure(fg_color=("#ea077c", "white") ,hover_color = ("green","white"))

# # Beam button
# root_ctk.beam_btn = customtkinter.CTkButton(root_ctk.left_sidebar_frame, state="disabled", text="Beam", font=customtkinter.CTkFont(weight="bold"))
# root_ctk.beam_btn.grid(row=7, column=0, padx=10, pady=5, sticky="nsew")
# root_ctk.beam_btn.configure(fg_color=("#ea077c", "white") ,hover_color = ("green","white"))

# # Hill Climbing button
# root_ctk.hill_btn = customtkinter.CTkButton(root_ctk.left_sidebar_frame, state="disabled", text="Hill Climbing", font=customtkinter.CTkFont(weight="bold"))
# root_ctk.hill_btn.grid(row=8, column=0, padx=10, pady=5, sticky="nsew")
# root_ctk.hill_btn.configure(fg_color=("#ea077c", "white") ,hover_color = ("green","white"))

# # A* button
# root_ctk.a_btn = customtkinter.CTkButton(root_ctk.left_sidebar_frame, state="disabled", text="A*", font=customtkinter.CTkFont(weight="bold"))
# root_ctk.a_btn.grid(row=9, column=0, padx=10, pady=5, sticky="nsew")
# root_ctk.a_btn.configure(fg_color=("#ea077c", "white") ,hover_color = ("green","white"))

In [9]:
# Remarks Frame Element
# Total nodes count
root_ctk.label_remarks = customtkinter.CTkLabel(root_ctk.remarks_frame, text = " ", text_color ="red", font=customtkinter.CTkFont(size=15, weight="bold"))
root_ctk.label_remarks.grid(row=0 , column=0, columnspan=2, padx=10, pady=(10, 0), sticky="ew")

# Output Frame Elements
#===================================================================

# Total nodes count
root_ctk.label_nodes_count = customtkinter.CTkLabel(root_ctk.output_frame, text = "No. of Nodes:   ", font=customtkinter.CTkFont(size=13, weight="bold"))
root_ctk.label_nodes_count.grid(row=0 , column=0, padx=10, pady=(10, 0), sticky="w")

# Total paths count
root_ctk.label_path_count = customtkinter.CTkLabel(root_ctk.output_frame, text = "No. of Paths:   ", font=customtkinter.CTkFont(size=13, weight="bold"))
root_ctk.label_path_count.grid(row=1 , column=0, padx=10, pady=(10, 0), sticky="w")

In [10]:
# Count Frame Elements
#===================================================================

# On search label
root_ctk.label_nodes = customtkinter.CTkLabel(root_ctk.count_frame, text = "On Search", font=customtkinter.CTkFont(size=16, weight="bold"))
root_ctk.label_nodes.grid(row=0 , column=0, columnspan=2, padx=10, pady=(10, 0), sticky="ew")

# Enques count text
root_ctk.label_enqueues = customtkinter.CTkLabel(root_ctk.count_frame, text = "Enques Count: ", font=customtkinter.CTkFont(size=13, weight="bold"))
root_ctk.label_enqueues.grid(row=1 , column=0, padx=10, pady=(10, 0), sticky="w")

# Path count text
root_ctk.label_path_cost = customtkinter.CTkLabel(root_ctk.count_frame, text = "Path Cost: ", font=customtkinter.CTkFont(size=13, weight="bold"))
root_ctk.label_path_cost.grid(row=1 , column=1, padx=10, pady=(10, 0), sticky="w")

In [11]:
# Bottom Frame Elements
#===================================================================

# Speed Slider
root_ctk.label_search = customtkinter.CTkLabel(root_ctk.bottom_frame, text = "Speed", font=customtkinter.CTkFont(size=14,weight="bold"))
root_ctk.label_search.grid(row=1 , column=0, padx=(10,5), pady=(10, 20), sticky="nsew")

root_ctk.sldr_speed = customtkinter.CTkSlider(root_ctk.bottom_frame, from_=2, to=4, number_of_steps=4, command=set_speed_lbl)
root_ctk.sldr_speed.grid(row=1, column=1, padx=(0,0), pady=(10, 20), sticky="nsew")
root_ctk.sldr_speed.set(2)
root_ctk.sldr_speed.configure(fg_color=("#ea077c", "white"))

root_ctk.label_sldr = customtkinter.CTkLabel(root_ctk.bottom_frame, text = str(speed) + "sec", font=customtkinter.CTkFont(size=14, weight="bold"))
root_ctk.label_sldr.grid(row=1 , column=2, padx=(0,10), pady=(10, 20), sticky="nsew")

# Start Button
root_ctk.btn_start = customtkinter.CTkButton(root_ctk.bottom_frame, text="Start", font=customtkinter.CTkFont(weight="bold"), command=play_clicked)
root_ctk.btn_start.grid(row=1, column=3, padx=(0,5), pady=(10,20), sticky="nsew")
root_ctk.btn_start.configure(fg_color=("#ea077c", "white") ,hover_color = ("green","white"))

# Pause Button
root_ctk.btn_pause = customtkinter.CTkButton(root_ctk.bottom_frame, state="disabled", text="Pause", font=customtkinter.CTkFont(weight="bold"), command=pause_clicked)
root_ctk.btn_pause.grid(row=1, column=4, padx=(5,10), pady=(10,20), sticky="nsew")
root_ctk.btn_pause.configure(fg_color=("#ea077c", "white") ,hover_color = ("green","white"))

In [12]:

fig, ax = plt.subplots(figsize=(6, 4))
canvas = FigureCanvasTkAgg(fig, master = root_ctk.graph_frame)
canvas_widget = canvas.get_tk_widget()
canvas_widget.grid(row=0, column=0, rowspan=8, columnspan=2, sticky="nsew")

# Display initial graph
G, custom_positions = set_fixed_graph(G)
display_init_graph(G, custom_positions)
embed_matplotlib_figure_to_tkinter()


In [ ]:
# Show GUI
root_ctk.mainloop()